In [ ]:
A faire: 
- filtrer ne prendre les DRG qu'avec au moins 10 observations, il faut filtrer également la première liste    
- préparer les dictionnaires? 

In [1]:
import pandas as pd
import numpy as np

In [30]:
from sklearn.preprocessing import OneHotEncoder,PolynomialFeatures, MaxAbsScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn import preprocessing
import dill
import math
from sklearn import base

In [52]:
from sklearn.base import BaseEstimator, RegressorMixin


In [ ]:
from sklearn.utils import shuffle

In [ ]:
from timeit import default_timer as timer


In [3]:
inpatients=pd.read_csv('prepared_data/Inpatient.csv')

In [4]:
local=pd.read_csv('prepared_data/Output_local_basis.csv')

In [5]:
hospitals=pd.read_csv('prepared_data/hospitals.csv')

In [11]:
temp=local.merge(hospitals, left_on='zipcode2017', right_on='zipcode')

In [18]:
full_inner=inpatients.merge(temp, left_on='prov_id', right_on='Facility ID')

In [21]:
categorical_columns = ['DRG','hrrnum','hosp_ownership','prov_id']
numeric_columns=['average_AGI_c', 'average_wage_c','per_over_65','per_ba','per_h','Score','total_discharges' ]

In [82]:
y = full_inner['average_covered_charges']
X = full_inner.loc[:,categorical_columns+ numeric_columns]

### Model 1: with hospital fixed effects


In [96]:
#Split the data


X, y = shuffle(X, y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)

In [97]:
categorical_columns_0 = ['DRG', 'prov_id']

In [99]:
start = timer()
features = ColumnTransformer([
    ('categorical', OneHotEncoder(handle_unknown='ignore'), categorical_columns_1)])

est_0 = Pipeline([
    ('features', features),
    ('estimator', LinearRegression())])

est_0.fit(X_train[categorical_columns_0], y_train, estimator__sample_weight=X_train['total_discharges'])
end = timer()

print(end-start)
print(f'R^2 score on train data: {est_0.score(X_train[categorical_columns_0], y_train)}')
print(f'R^2 score on test data: {est_0.score(X_test[categorical_columns_0], y_test)}')

y_pred=est_0.predict(X_test[categorical_columns_0])
print('average error rate on test set: ', abs((y_pred-y_test)/y_test).mean())
print('average error  on test set: ', abs((y_pred-y_test)).mean())

1.0880236060002062
R^2 score on train data: 0.7896896242371354
R^2 score on test data: 0.7993248028903852
average error rate on test set:  0.36108078552439055
average error  on test set:  16263.713524567485


In [103]:
class ResidualEstimator(BaseEstimator, RegressorMixin):

    def __init__(self,n_estimators=100,min_samples_leaf=1000,max_features='auto'):
        self.n_estimators=n_estimators
        self.min_samples_leaf=min_samples_leaf
        self.max_features=max_features

    def fit(self, X, y):
        est_1=LinearRegression().fit(X, y)
        res = y-est_1.predict(X)
        est_2=RandomForestRegressor(min_samples_leaf=self.min_samples_leaf,
                                    max_features=self.max_features,
                                    n_estimators=self.n_estimators).fit(X, res)
        self.estimator_1_=est_1
        self.estimator_2_=est_2
        return self

    def predict(self, X):
        return self.estimator_1_.predict(X)+self.estimator_2_.predict(X)


In [104]:
categorical_columns_1 = ['DRG','hrrnum','hosp_ownership', 'prov_id']
numeric_columns_1=['average_AGI_c', 'average_wage_c','per_over_65','per_ba','per_h','total_discharges']

In [105]:
X_train.columns

Index(['DRG', 'hrrnum', 'hosp_ownership', 'prov_id', 'average_AGI_c',
       'average_wage_c', 'per_over_65', 'per_ba', 'per_h', 'Score',
       'total_discharges'],
      dtype='object')

In [106]:
start = timer()
features = ColumnTransformer([
    ('categorical', OneHotEncoder(handle_unknown='ignore'), categorical_columns_1),
('numeric', 'passthrough', numeric_columns_1)])

est_1 = Pipeline([
    ('features', features),
    ('estimator', ResidualEstimator(min_samples_leaf=200,n_estimators=100,max_features="auto"))])

est_1.fit(X_train[categorical_columns_1+numeric_columns_1], y_train
         # , estimator__sample_weight=X_train['total_discharges']
         )
end = timer()

print(end-start)
print(f'R^2 score on train data: {est_1.score(X_train[categorical_columns_1+numeric_columns_1], y_train)}')
print(f'R^2 score on test data: {est_1.score(X_test[categorical_columns_1+numeric_columns_1], y_test)}')

y_pred=est_1.predict(X_test[categorical_columns_1+numeric_columns_1])
print('average error rate on test set: ', abs((y_pred-y_test)/y_test).mean())
print('average error  on test set: ', abs((y_pred-y_test)).mean())

307.6619012350002
R^2 score on train data: 0.8025028902086997
R^2 score on test data: 0.8069089433466456
average error rate on test set:  0.3036911261571947
average error  on test set:  15206.334566853897


Results with model 1 and 100, 100 as parameters

438.5626369839997
R^2 score on train data: 0.8151548136977186
R^2 score on test data: 0.8187969428344577
average error rate on test set:  0.2917842504502843
average error  on test set:  14557.084383963018

In [107]:
dill.dump(est_1,open('prepared_data/est_1.pkd', 'wb'))  

### Model 2: without hospital fixed effects but with score


In [108]:
full_inner_w_spending=full_inner[full_inner.Footnote.isna()]

In [109]:
y_2 = full_inner_w_spending['average_covered_charges']
X_2 = full_inner_w_spending.loc[:,categorical_columns+ numeric_columns]

In [110]:
X_2, y_2 = shuffle(X_2, y_2)
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_2, y_2, test_size=0.15)

In [111]:
categorical_columns_2 = ['DRG','hrrnum','hosp_ownership']
numeric_columns_2=[ 'average_wage_c','per_over_65','Score','average_AGI_c','per_ba','per_h']

In [115]:
start = timer()
features = ColumnTransformer([
    ('categorical', OneHotEncoder(handle_unknown='ignore'), categorical_columns_2),
('numeric', 'passthrough', numeric_columns_2)])

est_2 = Pipeline([
    ('features', features),
    ('estimator', ResidualEstimator(min_samples_leaf=150,n_estimators=100,max_features="auto"))])

est_2.fit(X_train_2[categorical_columns_2+numeric_columns_2], y_train_2
         # , estimator__sample_weight=X_train['total_discharges']
         )
end = timer()

print(end-start)
print(f'R^2 score on train data: {est_2.score(X_train_2[categorical_columns_2+numeric_columns_2], y_train_2)}')
print(f'R^2 score on test data: {est_2.score(X_test_2[categorical_columns_2+numeric_columns_2], y_test_2)}')

y_pred_2=est_2.predict(X_test_2[categorical_columns_2+numeric_columns_2])
print('average error rate on test set: ', abs((y_pred_2-y_test_2)/y_test_2).mean())
print('average error  on test set: ', abs((y_pred_2-y_test_2)).mean())

244.5862197009992
R^2 score on train data: 0.7778251967345248
R^2 score on test data: 0.7854592549646853
average error rate on test set:  0.3358527444501266
average error  on test set:  16882.18693922944


In [116]:
dill.dump(est_2,open('prepared_data/est_2.pkd', 'wb'))  

### Model 3: without hospital fixed effects and without score


In [117]:
categorical_columns_3 = ['DRG','hrrnum','hosp_ownership']
numeric_columns_3=[ 'average_wage_c','per_over_65','average_AGI_c','per_ba','per_h']

In [118]:
start = timer()
features = ColumnTransformer([
    ('categorical', OneHotEncoder(handle_unknown='ignore'), categorical_columns_3),
('numeric', 'passthrough', numeric_columns_3)])

est_3 = Pipeline([
    ('features', features),
    ('estimator', ResidualEstimator(min_samples_leaf=100,n_estimators=200,max_features="auto"))])

est_3.fit(X_train[categorical_columns_3+numeric_columns_3], y_train
         # , estimator__sample_weight=X_train['total_discharges']
         )
end = timer()

print(end-start)
print(f'R^2 score on train data: {est_3.score(X_train[categorical_columns_3+numeric_columns_3], y_train)}')
print(f'R^2 score on test data: {est_3.score(X_test[categorical_columns_3+numeric_columns_3], y_test)}')

y_pred=est_3.predict(X_test[categorical_columns_3+numeric_columns_3])
print('average error rate on test set: ', abs((y_pred-y_test)/y_test).mean())
print('average error  on test set: ', abs((y_pred-y_test)).mean())

566.6292813309992
R^2 score on train data: 0.7680246767879524
R^2 score on test data: 0.7755138654174181
average error rate on test set:  0.3531908481102657
average error  on test set:  17168.51622624461


In [119]:
dill.dump(est_3,open('prepared_data/est_3.pkd', 'wb'))  

### Model 4: combination of models 1 and 2

In [120]:
X_comb_test=np.concatenate((np.reshape(est_1.predict(X_test_2[categorical_columns_1 + numeric_columns_1]), (-1,1)),
                            np.reshape(est_2.predict(X_test_2[categorical_columns_2 + numeric_columns_2]), (-1,1))), axis=1)
X_comb_train=np.concatenate((np.reshape(est_1.predict(X_train_2[categorical_columns_1 + numeric_columns_1]), (-1,1)),
                             np.reshape(est_2.predict(X_train_2[categorical_columns_2 + numeric_columns_2]), (-1,1))), axis=1)


In [123]:
est_4 = RandomForestRegressor(max_depth=6, random_state=0)
est_4.fit(X_comb_train, y_train_2)

print('average error rate on the training set: ',abs((est_4.predict(X_comb_train)-y_train_2)/y_train_2).mean())
print('average error on the training set: ',abs((est_4.predict(X_comb_train)-y_train_2)).mean())

print(f'R^2 score using selected columns and transformers: {est_4.score(X_comb_test, y_test_2)}')

print('average error rate on the test set: ',abs((est_4.predict(X_comb_test)-y_test_2)/y_test_2).mean())
print('average error  on the test set: ',abs((est_4.predict(X_comb_test)-y_test_2)).mean())


average error rate on the training set:  0.2118386435463313
average error on the training set:  12709.234605909845
R^2 score using selected columns and transformers: 0.878684312330007
average error rate on the test set:  0.2104155487998252
average error  on the test set:  12838.28532852345


In [124]:
dill.dump(est_4,open('prepared_data/est_4.pkd', 'wb'))  

In [126]:
%%bash
ls

Models.ipynb
Visualizations.ipynb
prepared_data
raw_inputs


In [ ]:
%%bash
git add Models.ipynb prepared_data/est_1.pkd prepared_data/est_2.pkd prepared_data/est_3.pkd prepared_data/est_4.pkd